In [1]:
# #!COSAS POR HACER
# - Hacer script a partir del notebook
# - Añadir normalizacion

In [2]:
#!pip install netCDF4

In [3]:
import yaml

def load_config():
    with open("./config/params.yaml", "r") as f:
        return yaml.load(f, Loader=yaml.FullLoader)

In [3]:
import netCDF4 as nc
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import glob

In [4]:
PATH = '/home/samuel/Documents/local/projects/UPM/TFM/raw_data'

LONGITUDE = (18, 35)
LATITUDE = (10, 28)
LEVELS = (0, 8)

In [5]:
def extract_data(path, var_names, level):

    if level == 'upper':
        files = sorted(glob.glob(path + '/*-*-upper.nc'))

    elif level == 'surface':
        files = sorted(glob.glob(path + '/*-*-surface.nc'))
    else:
        raise ValueError('level must be "upper" or "surface"')

    # Dictionary to store concatenated data
    data = {var: [] for var in var_names}

    # Itearte over each file
    for file in files:
        with nc.Dataset(file, 'r') as nc_file:
            for var in var_names:
                if var == 'time':
                    data[var].append(nc_file.variables[var][:])
                else:
                    if level == 'upper':
                        data[var].append(nc_file.variables[var][:,LEVELS[0]:LEVELS[1]+1, LATITUDE[0]:LATITUDE[1]+1, LONGITUDE[0]:LONGITUDE[1]+1])
                    elif level == 'surface':
                        data[var].append(nc_file.variables[var][:,LATITUDE[0]:LATITUDE[1]+1, LONGITUDE[0]:LONGITUDE[1]+1])

    # Concatenate data for each variable
    for var in var_names:
        data[var] = np.concatenate(data[var], axis=0)
        data[var] = data[var].filled(np.nan)

    return data


In [6]:
upper_var_names = ['time', 'u', 'v', 'q', 't', 'd', 'z', 'w', 'vo']
surface_var_names = ['time', 'u100', 'v100', 'u10', 'v10', 'd2m', 't2m', 'z', 'msl']

upper_data = extract_data(PATH, upper_var_names, 'upper')
surface_data = extract_data(PATH, surface_var_names, 'surface')


## Save constants

In [7]:

with nc.Dataset('/home/samuel/Documents/local/projects/UPM/TFM/raw_data/2021-02-upper.nc', 'r') as nc_file:
    np.save('./config/latitude.npy', nc_file.variables["latitude"][:].filled(np.nan).astype(np.float16))
    np.save('./config/longitude.npy', nc_file.variables["longitude"][:].filled(np.nan).astype(np.float16))
    np.save('./config/level.npy', nc_file.variables["level"][:].filled(np.nan).astype(np.uint16))
    
with nc.Dataset('/home/samuel/Documents/local/projects/UPM/TFM/raw_data/2021-02-surface.nc', 'r') as nc_file:
    np.save('./config/land_sea_mask.npy', nc_file.variables["lsm"][:].filled(np.nan).astype(np.float32))

In [8]:
np.save('./data/time.npy', upper_data['time'].astype('uint32'))

del upper_data['time']
del surface_data['time']

upper_var_names.remove('time')
surface_var_names.remove('time')

## Reducción de tamaño
Se baja la precisión de las matrices numpy a float32, float16 y uint32

In [9]:
print("Upper variables\n")
for var in upper_data.keys():
    print(var, upper_data[var].dtype)

print("\nSurface variables\n")
for var in surface_data.keys():
    print(var, surface_data[var].dtype)

Upper variables

u float64
v float64
q float64
t float64
d float64
z float64
w float64
vo float64

Surface variables

u100 float64
v100 float64
u10 float64
v10 float64
d2m float64
t2m float64
z float64
msl float64


In [10]:
upper_total = 0
surface_total = 0

for var in upper_data.keys():
    upper_total += upper_data[var].nbytes/1e6
print("Upper: ", upper_total, "MB")

for var in surface_data.keys():
    surface_total += surface_data[var].nbytes/1e6
print("Surface: ", surface_total, "MB")

print("\nTotal size: ", upper_total + surface_total, "MB")

Upper:  6907.327488000001 MB
Surface:  767.4808320000001 MB

Total size:  7674.808320000001 MB


In [11]:
for var in upper_data.keys():
    upper_data[var] = upper_data[var].astype(np.float32)

for var in surface_data.keys():
    surface_data[var] = surface_data[var].astype(np.float32)


In [12]:
upper_total = 0
surface_total = 0

for var in upper_var_names:
    upper_total += upper_data[var].nbytes/1e6
print("Upper: ", upper_total, "MB")

for var in surface_var_names:
    surface_total += surface_data[var].nbytes/1e6
print("Surface: ", surface_total, "MB")

print("\nTotal size: ", upper_total + surface_total, "MB")

Upper:  3453.6637440000004 MB
Surface:  383.74041600000004 MB

Total size:  3837.4041600000005 MB


## Save data

In [13]:

surface_data_arrays = []
upper_data_arrays = []

for var in upper_data.keys():
    print(var, upper_data[var].shape)
    upper_data_arrays.append(upper_data[var])

for var in surface_data.keys():
    print(var, surface_data[var].shape)
    surface_data_arrays.append(surface_data[var])


combined_upper_data = np.stack(upper_data_arrays, axis=-1)
combined_surface_data = np.stack(surface_data_arrays, axis=-1)

del surface_data_arrays
del upper_data_arrays
del upper_data
del surface_data

# Imprime la forma del arreglo combinado
print(combined_upper_data.shape)
print(combined_surface_data.shape)

u (35064, 9, 19, 18)
v (35064, 9, 19, 18)
q (35064, 9, 19, 18)
t (35064, 9, 19, 18)
d (35064, 9, 19, 18)
z (35064, 9, 19, 18)
w (35064, 9, 19, 18)
vo (35064, 9, 19, 18)
u100 (35064, 19, 18)
v100 (35064, 19, 18)
u10 (35064, 19, 18)
v10 (35064, 19, 18)
d2m (35064, 19, 18)
t2m (35064, 19, 18)
z (35064, 19, 18)
msl (35064, 19, 18)
(35064, 9, 19, 18, 8)
(35064, 19, 18, 8)


In [14]:
np.save('./data/surface.npy', combined_surface_data)
np.save('./data/upper.npy', combined_upper_data)

In [15]:
"""
current_path = os.path.join(PATH, '2022-01-upper.nc')

with nc.Dataset(current_path) as nc_file:

    # Coordenadas fijas
    lat_index = 26
    lon_index = 18

    # Extraer variables u y v (velocidad del viento)
    u = nc_file.variables['u'][:, 0, lat_index, lon_index]
    v = nc_file.variables['v'][:, 0, lat_index, lon_index]

    # Calcular la velocidad total del viento para cada timestamp
    velocidad_viento = np.sqrt(u**2 + v**2)

    # Preparar el gráfico
    plt.figure()
    plt.plot(velocidad_viento)
    plt.title('Velocidad del Viento a lo Largo del Tiempo en una Ubicación Fija')
    plt.xlabel('Timestamp')
    plt.ylabel('Velocidad del Viento')
    plt.show()
"""

"\ncurrent_path = os.path.join(PATH, '2022-01-upper.nc')\n\nwith nc.Dataset(current_path) as nc_file:\n\n    # Coordenadas fijas\n    lat_index = 26\n    lon_index = 18\n\n    # Extraer variables u y v (velocidad del viento)\n    u = nc_file.variables['u'][:, 0, lat_index, lon_index]\n    v = nc_file.variables['v'][:, 0, lat_index, lon_index]\n\n    # Calcular la velocidad total del viento para cada timestamp\n    velocidad_viento = np.sqrt(u**2 + v**2)\n\n    # Preparar el gráfico\n    plt.figure()\n    plt.plot(velocidad_viento)\n    plt.title('Velocidad del Viento a lo Largo del Tiempo en una Ubicación Fija')\n    plt.xlabel('Timestamp')\n    plt.ylabel('Velocidad del Viento')\n    plt.show()\n"